In [39]:
import os 
import json
import psycopg2

In [44]:
#establishing connection to db
conn = psycopg2.connect("host=10.0.0.4 port=5432 dbname=pid-portal user=postgres password=passwd") 
cur = conn.cursor()

In [43]:
cur.execute('SELECT * FROM "User"') #just simple select to check if connection is working
cur.fetchone()

InterfaceError: cursor already closed

In [42]:
conn.close()

# Vehicle positions


In [45]:
def digest(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #df = pd.read_json(file, lines=True)
    return data


In [46]:
entries = [digest(entry) for entry in os.scandir('../../data/') if entry.is_file()]

In [47]:
entries[0][2]['features'][4]['properties']


{'trip': {'cis_agency_name': 'ABOUT ME',
  'cis_id': '100166',
  'cis_number': 96,
  'cis_order': 5,
  'cis_parent_route_name': '166',
  'cis_real_agency_name': 'ABOUT ME',
  'cis_short_name': '166',
  'cis_vehicle_registration_number': 1935,
  'gtfs_route_id': 'L166',
  'gtfs_route_short_name': '166',
  'gtfs_trip_id': '166_292_181001',
  'id': '2019-08-12T19:09:00Z_100166_166_96',
  'start_cis_stop_id': 59501,
  'start_cis_stop_platform_code': 'B',
  'start_time': '21:09:00',
  'start_timestamp': '2019-08-12T19:09:00.000Z',
  'vehicle_type': 3,
  'wheelchair_accessible': True},
 'last_position': {'bearing': 75,
  'cis_last_stop_id': 59395,
  'cis_last_stop_sequence': 33,
  'delay': 269,
  'delay_stop_arrival': None,
  'delay_stop_departure': 284,
  'gtfs_last_stop_id': None,
  'gtfs_last_stop_sequence': None,
  'gtfs_next_stop_id': 'U63Z1P',
  'gtfs_next_stop_sequence': None,
  'gtfs_shape_dist_traveled': '16.8',
  'is_canceled': False,
  'lat': '50.15097',
  'lng': '14.51525',
  'or

In [52]:
i = 0
for line in entries[0]:
    for data in line['features']:
        cur.execute("""
                INSERT INTO public."Vehicle_position"(
                route_id, "timestamp", bearing, calculated_delay, speed, longitude, latitude, vehicle_id, trip_id, is_canceled)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                """,
               (data['properties']['trip']['gtfs_route_id'],data['properties']['trip']['start_timestamp'], 
                data['properties']['last_position']['bearing'], data['properties']['last_position']['delay'],
                data['properties']['last_position']['speed'], data['properties']['last_position']['lng'], data['properties']['last_position']['lat'],
                data['properties']['trip']['cis_vehicle_registration_number'], data['properties']['trip']['gtfs_trip_id'], data['properties']['last_position']['is_canceled']
               ))
        #filling the vehicle table from the position data
        cur.execute("""
        INSERT INTO public."Vehicle"(
	vehicle_id, vehicle_type, agency)
	VALUES (%s, %s, %s)
    ON CONFLICT (vehicle_id) DO UPDATE 
  SET vehicle_type = excluded.vehicle_type, 
  agency = excluded.agency
    ;
        """,
            (data['properties']['trip']['cis_vehicle_registration_number'], data['properties']['trip']['vehicle_type'], data['properties']['trip']['cis_agency_name'])
                   )
        i += 1
print('Executed {} entries for table Vehicle_position'.format(i))

Executed 8718 entries for table Vehicle_position


In [53]:
conn.commit()

# Stops

In [54]:
def get_bool_from_int_for_wheelchair(bool_val):
    if bool_val == 1:
        return "1"
    else:
        return "0"

In [55]:
def digest_stops(file):
    f = open(file, 'r')
    #data = [json.loads(line) for line in open(file, 'r')]
    data = json.load(f)
    return data

In [56]:
entries = [digest_stops(entry) for entry in os.scandir('/mnt/d/data_diplomka/stops') if entry.is_file()]

In [57]:
entries[0]['features'][5]['properties']

{'level_id': '',
 'location_type': 2,
 'parent_station': 'U1000S1',
 'platform_code': '',
 'stop_code': None,
 'stop_desc': None,
 'stop_id': 'U1000S1E3',
 'stop_lat': 50.12497,
 'stop_lon': 14.51587,
 'stop_name': 'Listova ul.',
 'stop_timezone': None,
 'stop_url': '',
 'wheelchair_boarding': 1,
 'zone_id': '',
 'create_batch_id': None,
 'created_at': '2019-09-26T03:00:21.913Z',
 'created_by': None,
 'update_batch_id': None,
 'updated_at': '2019-09-26T03:00:21.913Z',
 'updated_by': None}

In [61]:
i = 0
for a in entries[0]['features']:
    
    cur.execute("""
    INSERT INTO public."Stop"(
	stop_id, name, latitude, longitude, zone_id, wheelchair_acc)
	VALUES (%s, %s, %s, %s, %s, %s);
    """,(a['properties']['stop_id'], a['properties']['stop_name'], a['properties']['stop_lat'], a['properties']['stop_lon'], a['properties']['zone_id'], 
        get_bool_from_int_for_wheelchair(a['properties']['wheelchair_boarding'])
        ))
    i += 1
print('Executed {} entries for table Stop'.format(i))

Executed 1000 entries for table Stop


In [62]:
conn.commit()

# Routes

In [63]:
def digest_routes(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #data = json.load(f)
    return data

In [64]:
entries = [digest_routes(entry) for entry in os.scandir('/mnt/d/data_diplomka/routes_test') if entry.is_file()]

In [65]:
entries[1][0]['features'][4]['properties']

{'shape_dist_traveled': 0.23124,
 'shape_id': 'L1V2',
 'shape_pt_lat': 50.09432,
 'shape_pt_lon': 14.3907,
 'shape_pt_sequence': 5}

In [68]:
i = 0
for route in entries:
    for point in route[0]['features']:
        cur.execute("""
        INSERT INTO public."Route"(
        route_id, "order", distance, latitude, longitude, stop_id)
        VALUES (%s, %s, %s, %s, %s, %s);
        """, 
                    (point['properties']['shape_id'], point['properties']['shape_pt_sequence'], point['properties']['shape_dist_traveled'], point['properties']['shape_pt_lat'],
                    point['properties']['shape_pt_lon'], '')
                   )
        i += 1
        #this will update the route with stops by finding route points that are within 10cm from stop location
cur.execute("""
        UPDATE "Route"
        SET stop_id=subquery.stop_id
            FROM (
            select A.stop_id, B.route_id, B.order from "Stop" as A, "Route" as B where calculate_distance(A.latitude, A.longitude, B.latitude,B.longitude, 'K') < 0.01
            ) AS subquery
        WHERE "Route".route_id=subquery.route_id and "Route".order=subquery.order;
        """)
        
print('Executed {} entries for stops'.format(i))

Executed 785 entries for stops


In [69]:
conn.commit()

# Trip

In [70]:
def digest_trips(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #data = json.load(f)
    return data

In [71]:
entries = [digest_trips(entry) for entry in os.scandir('/mnt/d/data_diplomka/trips') if entry.is_file()]

In [72]:
entries[0][1]

{'bikes_allowed': 0,
 'block_id': '',
 'direction_id': 0,
 'exceptional': 0,
 'route_id': 'L9',
 'service_id': '1111100-1',
 'shape_id': 'L9V7',
 'trip_headsign': 'Spojovací',
 'trip_id': '9_99_190902',
 'trip_operation_type': 1,
 'trip_short_name': '',
 'wheelchair_accessible': 2}

In [75]:
for file in entries:
    for entry in file:
        cur.execute("""
        INSERT INTO public."Trip"(
	trip_id, route_id, shape_id, direction, bikes_allowed, headsign)
	VALUES (%s, %s, %s, %s, %s, %s);""", 
                    (entry['trip_id'], entry['route_id'], entry['shape_id'], get_bool_from_int_for_wheelchair(entry['direction_id']), get_bool_from_int_for_wheelchair(entry['bikes_allowed']),
                    entry['trip_headsign'])
                    )

In [76]:
conn.commit()